# 数据挖掘互评作业三：分类与预测
## 所选数据集及问题
1. 数据集：Hotel booking demand
> 该数据集包含城市酒店和度假酒店的预订信息，包括预订时间、停留时间，成人/儿童/婴儿人数以及可用停车位数量等信息。  
数据量：32列共12W数据。
2. 问题探索
> 基于这个数据集，进行以下问题的探索：  
利用Logistic预测酒店预订。

## 数据集预处理

In [25]:
import os
from google.colab import drive
drive.mount('/content/drive') # mount google drive
co_path = '/content/drive/My Drive/Colab Notebooks/dataMining'
os.chdir(co_path)
# os.listdir(co_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import math
import re
import sys
import csv
import os

### 导入hotel-booking数据并观察该数据的基本信息
Hotel booking demand数据集共有32列和119390行。

基于需要探索的问题是预测客户是否进行预订，所以去除对结果没有影响或影响不大的列，包括：
hotel, agent, company,reservation_status_date

In [0]:
hotel_data = pd.read_csv('hotel-booking/hotel_bookings.csv')

In [28]:
hotel_data.head(5)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [29]:
hotel_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

In [0]:
drop_columns=['agent','company','hotel','reservation_status_date']
hotel_data.drop(inplace=True, axis=1, labels=drop_columns)

检查筛选后的数据发现country有488个记录有缺失值，因此用country中最频繁的值替换缺失值。

In [31]:
print("The shape of data after drop nan:",hotel_data.shape)
hotel_data.isnull().sum()

The shape of data after drop nan: (119390, 28)


is_canceled                         0
lead_time                           0
arrival_date_year                   0
arrival_date_month                  0
arrival_date_week_number            0
arrival_date_day_of_month           0
stays_in_weekend_nights             0
stays_in_week_nights                0
adults                              0
children                            4
babies                              0
meal                                0
country                           488
market_segment                      0
distribution_channel                0
is_repeated_guest                   0
previous_cancellations              0
previous_bookings_not_canceled      0
reserved_room_type                  0
assigned_room_type                  0
booking_changes                     0
deposit_type                        0
days_in_waiting_list                0
customer_type                       0
adr                                 0
required_car_parking_spaces         0
total_of_spe

In [32]:
hotel_data.fillna(hotel_data.mode().iloc[0], inplace=True)
hotel_data.isnull().sum()

is_canceled                       0
lead_time                         0
arrival_date_year                 0
arrival_date_month                0
arrival_date_week_number          0
arrival_date_day_of_month         0
stays_in_weekend_nights           0
stays_in_week_nights              0
adults                            0
children                          0
babies                            0
meal                              0
country                           0
market_segment                    0
distribution_channel              0
is_repeated_guest                 0
previous_cancellations            0
previous_bookings_not_canceled    0
reserved_room_type                0
assigned_room_type                0
booking_changes                   0
deposit_type                      0
days_in_waiting_list              0
customer_type                     0
adr                               0
required_car_parking_spaces       0
total_of_special_requests         0
reservation_status          

## 数据挖掘
### 划分数据集
is_cancelled 表示客户是否预订了酒店，因此将其作为标签，其它列作为特征。

对于每条记录，如果客户取消预订，is_cancelled值为1，否则该值为0。

In [0]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [34]:
hotel_data.head(5)

,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status
0,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,0,Transient,0.0,0,0,Check-Out
1,0,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,0,Transient,0.0,0,0,Check-Out
2,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,0,Transient,75.0,0,0,Check-Out
3,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,0,Transient,75.0,0,0,Check-Out
4,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,0,Transient,98.0,0,1,Check-Out


In [0]:
# X 为特征，y为标签
X = hotel_data.iloc[:,1:]
y = hotel_data.iloc[:,0]

In [36]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 27 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   lead_time                       119390 non-null  int64  
 1   arrival_date_year               119390 non-null  int64  
 2   arrival_date_month              119390 non-null  object 
 3   arrival_date_week_number        119390 non-null  int64  
 4   arrival_date_day_of_month       119390 non-null  int64  
 5   stays_in_weekend_nights         119390 non-null  int64  
 6   stays_in_week_nights            119390 non-null  int64  
 7   adults                          119390 non-null  int64  
 8   children                        119390 non-null  float64
 9   babies                          119390 non-null  int64  
 10  meal                            119390 non-null  object 
 11  country                         119390 non-null  object 
 12  market_segment  

因为数据同时具有数值特征和非数值特征，而Logistics Regression只能处理数值特征，所以使用sklearn.OneHotEncoder将非数值特征转化为数值特征。

In [0]:
#Implementing Column Transformer
need_transfer=['meal','distribution_channel','reservation_status','country','arrival_date_month','market_segment',
 'deposit_type','customer_type', 'reserved_room_type','assigned_room_type' ]
transfer = make_column_transformer(
    (OneHotEncoder(),need_transfer), remainder = 'passthrough'
    )
X = transfer.fit_transform(X).toarray()

将数据集的80%作为训练集，剩余的20%作为测试集。

In [0]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [39]:
X_train.shape

(95512, 256)

### 非数值特征处理
将非数值特征转化为数值特征后数据的特征维数增加到256，需要对其进行降维。
这里借助PCA将维数降低到100。

首先对数据集进行标准化：

In [0]:
stander = StandardScaler()
X_train = stander.fit_transform(X_train)
X_test = stander.transform(X_test)

运行PCA进行数据降维：

In [0]:
pca = PCA(n_components = 100)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
explained_variance = pca.explained_variance_ratio_

In [46]:
print("降维后的训练数据:",X_train.shape,"\n测试数据:",X_test.shape)
print("降维后的训练数据:",X_train,"\n测试数据:",X_test)

降维后的训练数据: (95512, 100) 
测试数据: (23878, 100)
降维后的训练数据: [[-2.58841895  2.83177243  1.65959814 ...  3.48495792  3.19028405
  -4.56666862]
 [-2.08195869  2.27023207  1.23929555 ...  1.61713963  1.50233861
   0.36458959]
 [-1.40686196  0.10865911 -1.38800891 ... -0.00920726  0.40034101
   0.44572372]
 ...
 [-2.67067702  0.16136514  0.06007937 ...  0.44450836  0.06987147
  -0.15713206]
 [ 1.46811704 -0.64639521 -2.53100676 ... -0.20161318 -0.79635944
  -0.13417013]
 [ 3.87700603  1.20219044  2.80774935 ...  0.14619875 -0.27513721
  -0.06266735]] 
测试数据: [[ 0.84543042  0.66847389 -1.39778522 ... -0.31244218 -0.24405844
   0.19355898]
 [-0.33354791  0.87965001 -1.33706422 ... -0.17393029  0.29776985
   0.39341203]
 [ 5.29041033  0.68271342  1.47558555 ... -0.01813283  0.00704848
   0.0878079 ]
 ...
 [ 1.02149469 -4.44666557 -1.74825027 ...  0.49281145  0.59010616
   0.67248342]
 [ 0.85685458 -5.92768689  0.00792185 ...  0.16050438 -0.22772996
  -0.02679273]
 [-0.82309515  0.78084244 -1.16130349 

### Logistics Regression
使用sklearn的LogisticsRegression进行训练和测试
#### 训练

In [47]:
classifier = LogisticRegression(random_state = 0, max_iter=10000)
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

#### 测试及结果

In [0]:
y_pred = classifier.predict(X_test)

In [51]:
prediction = confusion_matrix(y_test, y_pred)
prediction

array([[14918,    16],
       [   16,  8928]])

In [56]:
accuracy = (prediction[0][0] + prediction[1][1]) * 100.0 / prediction.sum()
print("The final test accuracy is %.4f%%" % (accuracy))

The final test accuracy is 99.8660%


对训练集进行测试

In [57]:
train_pred = classifier.predict(X_train)
train_prediction = confusion_matrix(y_train,train_pred)
print(train_prediction)
train_accuracy = (train_prediction[0][0] + train_prediction[1][1]) * 100.0 / train_prediction.sum()
print("The final accuracy on training data is %.4f%%" % (train_accuracy))

[[60173    59]
 [   75 35205]]
The final accuracy on training data is 99.8597%


## 分析
数据挖掘大部分工作集中在对数据的”预处理“上，
如这里根据探索目标选取Hotel booking demand数据集
中部分列来进行Logistics 回归分析，不仅需要慎重选择
需要用到的特征，还需要对非数值特征进行处理。

训练后在训练集和测试集上都取得了99.8+%的准确率，说明
所选的特征相对还算准确。